<a href="https://colab.research.google.com/github/GumballWatterson00/HousePricingUsingBlendedRegressors/blob/master/HousePricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from google.colab import files
import matplotlib.pyplot as plt
import io
import seaborn as sns
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 

auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
link = 'https://drive.google.com/file/d/1TMplQY_UF3f0rZ0UMMUBFiBXu_jC6Y6M/view?usp=sharing'
id = link.split("/")[-2] 
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('Nhà.csv')    
df = pd.read_csv('Nhà.csv') 
df = df.drop(['Giấy tờ pháp lý', 'Số tầng', 'Dài', 'Rộng'], axis=1)

In [3]:
df = df.dropna()

In [4]:
df = df.drop(df.columns[0], axis='columns')

In [5]:
 def processing(columns, N, M): 
  result = []
  for i in columns:
    i = i.partition(' ')[0].replace(',', '.')
    if i.count('.') > 1:
      i = float('nan')
      result.append(i)
    else:
      if float(i) <= N or float(i) >= M:
        i = float('nan')
        result.append(i)
      else:
        result.append(float(i))
  return result

In [6]:
df['Giá/m2'] = processing(df['Giá/m2'].to_list(), 30, 200)
df['Diện tích'] = processing(df['Diện tích'].to_list(), 20, 100)

In [7]:
df = df.dropna()

In [8]:
df

,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Số phòng ngủ,Diện tích,Giá/m2
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",5 phòng,46.0,86.96
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",3 phòng,37.0,116.22
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",4 phòng,40.0,65.00
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",6 phòng,51.0,100.00
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",4 phòng,36.0,86.11
...,...,...,...,...,...,...,...,...
82489,2019-09-19,"Đường Xuân Thủy, Phường Dịch Vọng Hậu, Quận Cầ...",Quận Cầu Giấy,Phường Dịch Vọng Hậu,"Nhà ngõ, hẻm",3 phòng,40.0,77.50
82490,2019-08-26,"Đường Lê Đức Thọ, Phường Mỹ Đình 1, Quận Nam T...",Quận Nam Từ Liêm,Phường Mỹ Đình 1,Nhà phố liền kề,3 phòng,38.0,76.32
82491,2019-08-23,"Đường Hồ Tùng Mậu, Phường Phúc Diễn, Quận Bắc ...",Quận Bắc Từ Liêm,Phường Phúc Diễn,Nhà phố liền kề,3 phòng,38.0,81.58
82494,2019-08-05,"Đường Quan Hoa, Phường Quan Hoa, Quận Cầu Giấy...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà ngõ, hẻm",4 phòng,60.0,101.67


In [9]:
df.Timestamp = df['Ngày']
df.Timestamp = df.Timestamp.astype('datetime64')
ADDITIONAL = 1
df['Timestamp'] = ((df.Timestamp.dt.year * (10000 * ADDITIONAL)) + (df.Timestamp.dt.month * (100 * ADDITIONAL)) + ((df.Timestamp.dt.day) * ADDITIONAL))
df = df.drop('Ngày', axis=1)

def processing_bedrooms(columns, N): 
  result = []
  for i in columns:
    i = i.partition(' ')[0].replace(',', '.')
    if i in ['Nhiều', 'nhiều']:
      result.append(11) 
    elif i.count('.') > 1:
      i = float('nan')
      result.append(i)
    else:
      if float(i) < N:
        i = float('nan')
        result.append(i)
      else:
        result.append(float(i))
  return result

df['Số phòng ngủ'] = processing_bedrooms(df['Số phòng ngủ'].to_list(), 1)

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

or_res = df.drop('Địa chỉ', axis=1)
features_to_encode = ['Quận', 'Huyện', 'Loại hình nhà ở']
for feature in features_to_encode: 
  or_res = encode_and_bind(or_res, feature)

for feature in or_res.columns:
  if any(or_res[feature]) == False:
    or_res.drop(feature, axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [10]:
res = or_res.copy()

In [11]:
res.drop('Timestamp', axis=1, inplace=True)
res['Giá'] = res['Giá/m2'] * res['Diện tích'] * 1e6
res['Số phòng ngủ'] = np.log1p(res['Số phòng ngủ'])
res['Giá'] = np.log1p(res['Giá'])
res.drop('Giá/m2', axis=1, inplace=True)
res['Diện tích'] = np.log1p(res['Diện tích'])

In [12]:
res.describe()

,Số phòng ngủ,Diện tích,Quận_Huyện Chương Mỹ,Quận_Huyện Gia Lâm,Quận_Huyện Hoài Đức,Quận_Huyện Mê Linh,Quận_Huyện Phúc Thọ,Quận_Huyện Sóc Sơn,Quận_Huyện Thanh Oai,Quận_Huyện Thanh Trì,Quận_Huyện Thường Tín,Quận_Huyện Thạch Thất,Quận_Huyện Đan Phượng,Quận_Huyện Đông Anh,Quận_Quận Ba Đình,Quận_Quận Bắc Từ Liêm,Quận_Quận Cầu Giấy,Quận_Quận Hai Bà Trưng,Quận_Quận Hoàn Kiếm,Quận_Quận Hoàng Mai,Quận_Quận Hà Đông,Quận_Quận Long Biên,Quận_Quận Nam Từ Liêm,Quận_Quận Thanh Xuân,Quận_Quận Tây Hồ,Quận_Quận Đống Đa,Quận_Thị xã Sơn Tây,Huyện_Phường Biên Giang,Huyện_Phường Bách Khoa,Huyện_Phường Bùi Thị Xuân,Huyện_Phường Bưởi,Huyện_Phường Bạch Mai,Huyện_Phường Bạch Đằng,Huyện_Phường Bồ Đề,Huyện_Phường Chương Dương,Huyện_Phường Cát Linh,Huyện_Phường Cầu Diễn,Huyện_Phường Cầu Dền,Huyện_Phường Cống Vị,Huyện_Phường Cổ Nhuế 1,...,Huyện_Xã Nguyên Khê,Huyện_Xã Ngũ Hiệp,Huyện_Xã Ngọc Hồi,Huyện_Xã Ngọc Tảo,Huyện_Xã Phù Lỗ,Huyện_Xã Phương Trung,Huyện_Xã Sơn Đồng,Huyện_Xã Tam Hiệp,Huyện_Xã Thanh Liệt,Huyện_Xã Thanh Xuân,Huyện_Xã Tiền Phong,Huyện_Xã Tân Hội,Huyện_Xã Tân Lập,Huyện_Xã Tân Triều,Huyện_Xã Tả Thanh Oai,Huyện_Xã Tứ Hiệp,Huyện_Xã Uy Nỗ,Huyện_Xã Vân Canh,Huyện_Xã Vân Côn,Huyện_Xã Vân Nội,Huyện_Xã Võng La,Huyện_Xã Vĩnh Ngọc,Huyện_Xã Vĩnh Quỳnh,Huyện_Xã Vạn Phúc,Huyện_Xã Xuân Nộn,Huyện_Xã Yên Thường,Huyện_Xã Yên Viên,Huyện_Xã Đa Tốn,Huyện_Xã Đông Dư,Huyện_Xã Đông Hội,Huyện_Xã Đông La,Huyện_Xã Đông Mỹ,Huyện_Xã Đặng Xá,Huyện_Xã Đức Giang,Huyện_Xã Đức Thượng,Loại hình nhà ở_Nhà biệt thự,"Loại hình nhà ở_Nhà mặt phố, mặt tiền","Loại hình nhà ở_Nhà ngõ, hẻm",Loại hình nhà ở_Nhà phố liền kề,Giá
count,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,...,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000,73600.000000
mean,1.538547,3.743148,0.000041,0.002201,0.004891,0.000027,0.000014,0.000041,0.000557,0.014022,0.000014,0.000014,0.000149,0.000530,0.051957,0.019008,0.078967,0.129851,0.002962,0.144280,0.099402,0.051685,0.039620,0.164606,0.033030,0.162092,0.000041,0.000380,0.005734,0.000557,0.007418,0.016644,0.003777,0.010326,0.000870,0.003234,0.002079,0.001264,0.007228,0.004266,...,0.000027,0.000231,0.001073,0.000014,0.000014,0.000014,0.000014,0.001155,0.003641,0.000014,0.000014,0.000014,0.000122,0.002853,0.001671,0.001141,0.000027,0.001535,0.000027,0.000027,0.000068,0.000027,0.000380,0.000014,0.000014,0.000014,0.000014,0.000041,0.000082,0.000014,0.000435,0.000109,0.000163,0.000027,0.000014,0.004103,0.172269,0.802079,0.021549,22.013370
std,0.260042,0.272356,0.006384,0.046864,0.069767,0.005213,0.003686,0.006384,0.023596,0.117581,0.003686,0.003686,0.012224,0.023013,0.221941,0.136554,0.269690,0.336141,0.054344,0.351376,0.299203,0.221391,0.195065,0.370827,0.178716,0.368538,0.006384,0.019501,0.075504,0.023596,0.085811,0.127934,0.061343,0.101092,0.029476,0.056774,0.045547,0.035525,0.084712,0.065178,...,0.005213,0.015196,0.032745,0.003686,0.003686,0.003686,0.003686,0.033964,0.060234,0.003686,0.003686,0.003686,0.011058,0.053340,0.040846,0.033764,0.005213,0.039153,0.005213,0.005213,0.008242,0.005213,0.019501,0.003686,0.003686,0.003686,0.003686,0.006384,0.00

In [13]:
X = res.drop('Giá', axis=1).to_numpy()
Y = res['Giá'].to_numpy()

from sklearn import preprocessing

scaler = preprocessing.RobustScaler()
scaler.fit(X)
X = scaler.transform(X)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

print(np.shape(X_val))

(7360, 264)


In [14]:
# str_list = []
# for colname, colvalue in res.iteritems():
#     if type(colvalue[1]) == str:
#          str_list.append(colname)
# num_list = res.columns.difference(str_list) 
# house_num = res[num_list]
# f, ax = plt.subplots(figsize=(16, 12))
# plt.title('Pearson Correlation of features')
# sns.heatmap(house_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="cubehelix", linecolor='k', annot=True)

In [69]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [16]:
param_init = {
    "max_depth": 5, 
    "n_estimators": 3000, 
    "learning_rate": 0.25, 
    "subsample": 0.5,
    "colsample_bytree": 0.7,
    "min_child_weight": 1.5,
    "reg_alpha": 0.75,
    "reg_lambda": 0.4,
    "seed": 42,
}

param_fit = {
    "eval_metric": "rmse",
    "early_stopping_rounds": 500,
    "verbose": 200,
    "eval_set": [(X_val, Y_val)]
}

xgb_model = xgboost.XGBRegressor(**param_init)
xgb_model.fit(X_train, Y_train, **param_fit)


[23:34:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:16.1395
Will train until validation_0-rmse hasn't improved in 500 rounds.
[200]	validation_0-rmse:0.24212
[400]	validation_0-rmse:0.240322
[600]	validation_0-rmse:0.240051
[800]	validation_0-rmse:0.239709
[1000]	validation_0-rmse:0.239607
[1200]	validation_0-rmse:0.23986
Stopping. Best iteration:
[831]	validation_0-rmse:0.239419



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.25, max_delta_step=0,
             max_depth=5, min_child_weight=1.5, missing=None, n_estimators=3000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.75, reg_lambda=0.4, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.5, verbosity=1)

In [27]:
Y_pred_xgb = xgb_model.predict(X_test)
(1- (abs(np.expm1(Y_pred_xgb) - np.expm1(Y_test)) / np.expm1(Y_test)).mean()) * 100

81.37975787905678

In [33]:
rf = RandomForestRegressor(n_estimators=2000,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)
rf.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features=None, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=None, oob_score=True,
                      random_state=42, verbose=0, warm_start=False)

In [34]:
Y_pred_rf = rf.predict(X_test)
(1- (abs(np.expm1(Y_pred_rf) - np.expm1(Y_test)) / np.expm1(Y_test)).mean()) * 100

80.460693478554

In [35]:
gbr = GradientBoostingRegressor(n_estimators=3000,
                                learning_rate=0.25,
                                max_depth=5,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)
gbr.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.25, loss='huber',
                          max_depth=5, max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [36]:
Y_pred_gbr = gbr.predict(X_test)
(1- (abs(np.expm1(Y_pred_gbr) - np.expm1(Y_test)) / np.expm1(Y_test)).mean()) * 100

81.52299834922161

In [70]:
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
kf = KFold(n_splits=12, random_state=42, shuffle=True)
ridge = RidgeCV(ridge_alphas, cv=kf)
ridge.fit(X_train, Y_train)

RidgeCV(alphas=array([1.0e-15, 1.0e-10, 1.0e-08, 9.0e-04, 7.0e-04, 5.0e-04, 3.0e-04,
       1.0e-04, 1.0e-03, 5.0e-02, 1.0e-02, 1.0e-01, 3.0e-01, 1.0e+00,
       3.0e+00, 5.0e+00, 1.0e+01, 1.5e+01, 1.8e+01, 2.0e+01, 3.0e+01,
       5.0e+01, 7.5e+01, 1.0e+02]),
        cv=KFold(n_splits=12, random_state=42, shuffle=True),
        fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
        store_cv_values=False)

In [72]:
Y_pred_ridge = ridge.predict(X_test)
(1- (abs(np.expm1(Y_pred_ridge) - np.expm1(Y_test)) / np.expm1(Y_test)).mean()) * 100

80.1008470856304

In [ ]:
Y_pred_final = (0.25 * np.expm1(xgb_model.predict(X_test)) +
                0.05 * np.expm1(rf.predict(X_test)) +
                0.65 * np.expm1(gbr.predict(X_test)) +
                0.05 * np.exmp1(ridge.predict(X_test)))